In [1]:
import numpy as np
import tensorflow as tf
import math
import random
from os import listdir
import codecs
import nltk
import data
from gensim.models import Word2Vec

In [2]:
vocab_size = 324
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'

1659


In [3]:
x, y = data.sample(d, 5)

In [4]:
hidden_size = 100
num_steps = 10
learning_rate = 0.01
num_runs_between_logging = 3000
epoch = 6000

In [5]:
Wxh = tf.Variable(tf.random_uniform([vocab_size, hidden_size])) 
Whh = tf.Variable(tf.random_uniform([hidden_size, hidden_size]))
bh = tf.Variable(tf.zeros([hidden_size]))
Why = tf.Variable(tf.random_uniform([hidden_size, vocab_size])) 
by = tf.Variable(tf.zeros([vocab_size]))
U_z = tf.Variable(tf.random_uniform([vocab_size, hidden_size]))
W_z = tf.Variable(tf.random_uniform([hidden_size, hidden_size]))
U_r = tf.Variable(tf.random_uniform([vocab_size, hidden_size]))
W_r = tf.Variable(tf.random_uniform([hidden_size, hidden_size]))

In [10]:
dd = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32)
inputs = tf.split(0, num_steps, dd)
# inputs = [tf.placeholder(tf.float32, [1, vocab_size]) for _ in xrange(num_steps)]
keep_prob = tf.placeholder(tf.float32)
outputs = {}
hs = {}
hs[-1] = tf.placeholder(tf.float32, [1, hidden_size])
# GRU UNIT
for i in range(len(inputs)):
    r = tf.nn.softmax(tf.matmul(inputs[i], U_r) + tf.matmul(hs[i-1], W_r))
    z = tf.nn.softmax(tf.matmul(inputs[i], U_z) + tf.matmul(hs[i-1], W_z))
    h = tf.nn.tanh(tf.matmul(inputs[i], Wxh) + tf.matmul(hs[i - 1] * r, Whh))
    hs[i] = tf.mul(tf.sub(tf.ones_like(z), z), h) + tf.mul(z , hs[i - 1])
#     hs[i]      = tf.nn.tanh(tf.matmul(inputs[i], Wxh) + tf.matmul(hs[i-1], Whh) + bh)
    hs_drop = tf.nn.dropout(hs[i], keep_prob)
    outputs[i] = tf.nn.softmax(tf.matmul(hs_drop, Why) + by)

tt = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32)
truths = tf.split(0, num_steps, tt)
# truths = [tf.placeholder(tf.float32, [1, vocab_size]) for i in xrange(num_steps)]
cross_entropies = [-tf.reduce_sum(truths[i]*tf.log(outputs[i])) for i in xrange(num_steps)]  # These operations act element-wise.
cross_entropy = tf.add_n(cross_entropies)
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(cross_entropy)

ValueError: Cannot infer num from shape (?, 324)

In [7]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [8]:
def hallucinate(sess, seed_ix, num_chars):
    ix = seed_ix
    ixes = [ix]
    hallucination_h_state = np.zeros([1, hidden_size]) 
    for i in range(num_chars):
        x_in = [[0] * vocab_size]
        x_in[0][ix] = 1

        feed_dict={inputs[0]: x_in}
        feed_dict.update({hs[-1]:hallucination_h_state})
        feed_dict.update({keep_prob: 1.0})

        output, hallucination_h_state = sess.run((outputs[0],hs[0]), feed_dict=feed_dict)
        probs = output[0]
        ix = np.random.choice(range(len(probs)), p=probs)
        ixes.append(ix)
    hallucination=' '.join([id_to_token[ix] for ix in ixes])
    print hallucination

In [9]:
losses = []
iterations = 0
ix = 0
h_state = np.zeros([1, hidden_size])
smooth_loss = -np.log(1.0/vocab_size)*num_steps # loss at iteration 0

In [14]:
def RunModel():
    global ix,losses,iterations,h_state,smooth_loss
    run = 0
    while True:
        letters=[]
        if run > epoch:
            break
        x, y = data.sample(d, num_steps)
#         feed_dict={inputs[i]: x[i].reshape(1, -1) for i in range(num_steps)}
        feed_dict = {dd: x}
#         feed_dict.update({truths[i]: y[i].reshape(1, -1) for i in range(num_steps)})
        feed_dict.update({tt : y})
        feed_dict.update({hs[-1]:h_state})
        feed_dict.update({keep_prob: 0.8})

        _, h_state, loss_out = sess.run((train_step, hs[num_steps-1], cross_entropy), feed_dict=feed_dict)
        smooth_loss = smooth_loss * 0.999 + loss_out * 0.001
        ix += num_steps
        #        letters.append(data[ix])
        run += 1
        if run % num_runs_between_logging == 0:   
            iterations += 1
            #        print "train: ", ''.join(letters)
            print "========== Iteration ", run, " Loss: ", smooth_loss, " =============="
            losses.append(smooth_loss)
#             hallucinate(sess, random.randint(0,vocab_size-1), 60)
RunModel()

========== Iteration  3000  Loss:  37.1425059609  ==============
========== Iteration  6000  Loss:  25.8454689933  ==============


In [101]:
hallucinate(sess, random.randint(0,vocab_size-1), 150)

sky . 
 And the autumn breezes sing . 
 
 And the bee-hive casts its swarm ; 
 Slipt its golden clasp , 
 With a waist and every place ? 
 Let , when her zone 
 Like three fit wines in pelteth . 
 Pearled with the cheek that doth not weary ? Where 's the eye , high-commission 'd : 
 White as does its blossoming ; 
 Cast on sunny bank its celled sleep ; 
 When the bee-hive casts its skin ; 
 Acorns ripe down-pattering , 
 Or the early April lark , 
 While the soundless earth hath lost ; 
 Sit thee there , and cloudward soar . 
 Distant harvest-carols clear ; 
 White-plum 'd lillies , and dew , 
 Beauties that the earth hath lost ; 
 She will bring thee , when rain pelteth . 
 The daisy


In [17]:
hallucinate(sess, random.randint(0,vocab_size-1), 150)

buds and the leaf , and alarm 
 Hatching buds and the reaped April lark , 
 Pleasure melteth , winged Fancy 's the ploughboy 's the Night doth meet the first 
 While the face 
 One would hear so very oft ? 
 When the mesh 
 Fades as how Fancy thy mind in the mist and dew , winged Fancy roam , 
 At a caw ' daughter , 
 Sit thee the field-mouse peep ; 
 Shaded hyacinth , in every flower 
 Like to its skin ; 
 Let the winged Fancy ! 
 All the golden dart forth , and with the ingle , 
 Meagre from her : 
 She will bring , 
 All the ploughboy 's wing doth meet the reaped corn ; 
 When the Night doth not weary ? Where 's the soundless earth hath burst ; 
 flower night
